<a href="https://colab.research.google.com/github/cristianmunoz1/ai4eng-proyecto/blob/main/11%20-%20preprocesado%20sin%20algunas%20columnas%20y%20departamento%20para%20test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Replicamos los recursos locales en el entorno virtual

### Descargamos los datos de la competencia al entorno de trabajo

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 84% 17.0M/20.1M [00:00<00:00, 27.8MB/s]
100% 20.1M/20.1M [00:01<00:00, 19.3MB/s]


### Descomprimimos el archivo

In [ ]:
!unzip udea*.zip > /dev/null

In [ ]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  50135751 test.csv
   692501  10666231 118025055 train.csv
  1286075  15528571 172877479 total


## Importe de librerías necesarias

Instalamos la librería Unidecode con el objetivo de manejar las cadenas de texto específicamente de la columna "ESTU_PRGM_ACADEMICO". Esto dado que debemos convertirla a minúsculas, eliminar tíldes y procesarlas todas de la manera más uniforme posible.

In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 8.9 MB/s eta 0:00:00


Usamos las librerías


*   **Unidecode**: Para manejar los datos de la columna de programa académico con la intención de estandarizar la escritura de las mismas.
*   **Pandas**: Para manejar los datos en un objeto Dataframe y poderlos manipular de manera sencilla
*   **numpy**: Para manejar los datos numéricos, ordenar y estandarizar vectores y columnas del dataframe
*   **OneHotEncoder y OrdinalEncoder de Scikitlearn**: Para procesar algunas columnas categóricas aplicándoles OneHot y otras de manera ordinal.



In [ ]:
import unidecode as ud
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

### Cargamos el csv como un dataframe de Pandas
Se guarda en la variable train para usarlo durante todo el preprocesado

In [ ]:
train = pd.read_csv("test.csv")
print ("shape of loaded dataframe", train.shape)

shape of loaded dataframe (296786, 12)


Observamos qué columnas tiene el dataframe de Pandas al momento de cargarlo.

In [ ]:
train.columns

Index(['Unnamed: 0', 'ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO',
       'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD',
       'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET',
       'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE',
       'ESTU_PAGOMATRICULAPROPIO'],
      dtype='object')

Observamos las primeras 5 filas del dataframe

In [ ]:
train.head()

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No


## One_hot encoding para la columna 'ESTU_PRGM_ACADEMICO'


#### Estrategia de preprocesado de 'ESTU_PRGM_ACADEMICO'

*   Observamos cuantas carreras únicas hay y cuales de estas son.
*   Las pasamos todas a minúsculas para que se agrupen algunas que eran las mismas pero escritas con diferentes combinaciones de mayúsculas y minúsculas.
*   Dejamos en un array de numpy solo las carreras que tengan más de 6000 apariciones en el Dataframe original. Con el objetivo de agrupar al rededor de 20 carreras diferentes.
*   A las demás carreras las agrupamos en un grupo 'otros' con el objetivo de reducir el ruido en los datos y la posible variabilidad que puedan presentar.
*   Aplicar OneHotEncoding a los nuevos datos en la columna.

In [ ]:
carreras_unicas = train['ESTU_PRGM_ACADEMICO'].unique()
len(carreras_unicas)

919

In [ ]:
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].apply(lambda x: ud.unidecode(x.lower()))

In [ ]:
train.head()

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,0,550236,20183,trabajo social,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,1,98545,20203,administracion comercial y de mercadeo,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,2,499179,20212,ingenieria mecatronica,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,3,782980,20195,contaduria publica,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,4,785185,20212,administracion de empresas,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No


In [ ]:
prgm_acads = ['administracion de empresas', 'derecho', 'contaduria publica',
       'psicologia', 'ingenieria industrial', 'ingenieria civil', 'medicina',
       'ingenieria de sistemas', 'licenciatura en pedagogia infantil',
       'ingenieria ambiental', 'trabajo social', 'enfermeria', 'arquitectura',
       'administracion en salud ocupacional', 'negocios internacionales',
       'economia', 'ingenieria mecanica', 'ingenieria electronica',
       'comunicacion social', 'administracion financiera']

In [ ]:
len(prgm_acads)

20

In [ ]:
array = [element for element in train['ESTU_PRGM_ACADEMICO'] if element not in prgm_acads ]
array

condition = train['ESTU_PRGM_ACADEMICO'].isin(array)
train.loc[condition, 'ESTU_PRGM_ACADEMICO'] = "otros"
train.head()

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,0,550236,20183,trabajo social,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,1,98545,20203,otros,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,2,499179,20212,otros,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,3,782980,20195,contaduria publica,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,4,785185,20212,administracion de empresas,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No


Los programas académicos quedan entonces de la siguiente manera después del procesado de preparación para aplicarles el OneHot Encoding.

In [ ]:
train['ESTU_PRGM_ACADEMICO'].value_counts()

,count
ESTU_PRGM_ACADEMICO,
otros,107041
administracion de empresas,33008
derecho,22984
contaduria publica,22103
psicologia,21256
ingenieria industrial,15228
ingenieria civil,9079
medicina,7915
ingenieria de sistemas,7655


In [ ]:
encode = OneHotEncoder()

codificadorPrg = encode.fit_transform(train[['ESTU_PRGM_ACADEMICO']])

arreglo = pd.DataFrame(codificadorPrg.toarray(), columns = encode.categories_)
print(arreglo)
train.drop('ESTU_PRGM_ACADEMICO', axis = 1)

train = pd.concat([train, arreglo], axis = 'columns')
train['ESTU_PRGM_ACADEMICO']

       administracion de empresas administracion en salud ocupacional  \
0                             0.0                                 0.0   
1                             0.0                                 0.0   
2                             0.0                                 0.0   
3                             0.0                                 0.0   
4                             1.0                                 0.0   
...                           ...                                 ...   
296781                        1.0                                 0.0   
296782                        0.0                                 0.0   
296783                        0.0                                 0.0   
296784                        0.0                                 0.0   
296785                        0.0                                 0.0   

       administracion financiera arquitectura comunicacion social  \
0                            0.0          0.0         

,ESTU_PRGM_ACADEMICO
0,trabajo social
1,otros
2,otros
3,contaduria publica
4,administracion de empresas
...,...
296781,administracion de empresas
296782,derecho
296783,derecho
296784,otros


Borramos la columna 'ESTU_PRGM_ACADEMICO' ya que no sirve para nada ahora que aplicamos el OneHot encoding a la misma.

In [ ]:
train.shape

(296786, 33)

In [ ]:
train = train.drop('ESTU_PRGM_ACADEMICO', axis = 1)

#### Dataset con preprocesado de la columna 'ESTU_PRGM_ACADEMICO'

In [ ]:
train.head()

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,...,"(ingenieria de sistemas,)","(ingenieria electronica,)","(ingenieria industrial,)","(ingenieria mecanica,)","(licenciatura en pedagogia infantil,)","(medicina,)","(negocios internacionales,)","(otros,)","(psicologia,)","(trabajo social,)"
0,0,550236,20183,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,98545,20203,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,499179,20212,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,782980,20195,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,785185,20212,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Categorización ordinal de las columnas categóricas ordinales

In [ ]:
train.columns

Index([                            'Unnamed: 0',
                                           'ID',
                                      'PERIODO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                           'FAMI_TIENEINTERNET',
                          'FAMI_EDUCACIONPADRE',
                          'FAMI_EDUCACIONMADRE',
                     'ESTU_PAGOMATRICULAPROPIO',
                ('administracion de empresas',),
       ('administracion en salud ocupacional',),
                 ('administracion financiera',),
                              ('arquitectura',),
                       ('comunicacion social',),
                        ('contaduria publica',),
                                   ('derecho',),
                                  ('economia',),
                                ('enfermeria',),
                    

## Quitarle los valores nulos a todas las columnas categóricas ordinales


### Conteo de valores nulos

In [ ]:
train['FAMI_EDUCACIONMADRE'].value_counts()

,count
FAMI_EDUCACIONMADRE,
Secundaria (Bachillerato) completa,60649
Primaria incompleta,42149
Técnica o tecnológica completa,38571
Educación profesional completa,36477
Secundaria (Bachillerato) incompleta,34566
Primaria completa,24399
Postgrado,20049
Técnica o tecnológica incompleta,11650
Educación profesional incompleta,9647


In [ ]:
print(f"Cantidad nulos valor matrícula = {train['ESTU_VALORMATRICULAUNIVERSIDAD'].isna().sum()}")
print(f"Cantidad nulos horas semanales trabaja = {train['ESTU_HORASSEMANATRABAJA'].isna().sum()}")
print(f"Cantidad nulos estrato vivienda = {train['FAMI_ESTRATOVIVIENDA'].isna().sum()}")
print(f"Cantidad nulos educación padre = {train['FAMI_EDUCACIONPADRE'].isna().sum()}")
print(f"Cantidad nulos educación madre = {train['FAMI_EDUCACIONMADRE'].isna().sum()}")

Cantidad nulos valor matrícula = 2723
Cantidad nulos horas semanales trabaja = 13379
Cantidad nulos estrato vivienda = 13795
Cantidad nulos educación padre = 9993
Cantidad nulos educación madre = 10223


### Cambio de valores nulos por 'no info'

In [ ]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'] = train['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna("no info")
train['ESTU_HORASSEMANATRABAJA'] = train['ESTU_HORASSEMANATRABAJA'].fillna("no info")
train['FAMI_ESTRATOVIVIENDA'] = train['FAMI_ESTRATOVIVIENDA'].fillna("no info")
train['FAMI_EDUCACIONPADRE'] = train['FAMI_EDUCACIONPADRE'].fillna("no info")
train['FAMI_EDUCACIONMADRE'] = train['FAMI_EDUCACIONMADRE'].fillna("no info")

## Codificación de valores categóricos ordinales.

Agrupamos todas las categorías de las variables categóricas ordinales en un arreglo con el objetivo de aplicar la codificación con el OrdinalEncoder.

In [ ]:
categories = [
    ['no info', 'No pagó matrícula', 'Menos de 500 mil', 'Entre 500 mil y menos de 1 millón',
     'Entre 1 millón y menos de 2.5 millones', 'Entre 2.5 millones y menos de 4 millones',
     'Entre 4 millones y menos de 5.5 millones', 'Entre 5.5 millones y menos de 7 millones',
     'Más de 7 millones'],  # ESTU_VALORMATRICULAUNIVERSIDAD
    ['no info', '0', 'Menos de 10 horas', 'Entre 11 y 20 horas', 'Entre 21 y 30 horas', 'Más de 30 horas'],  # ESTU_HORASSEMANATRABAJA
    ['no info', 'Sin Estrato', 'Estrato 1', 'Estrato 2', 'Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 6'],  # FAMI_ESTRATOVIVIENDA
    ['no info', 'Ninguno', 'No sabe', 'No Aplica', 'Primaria incompleta', 'Primaria completa',
     'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
     'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa', 'Postgrado'],  # FAMI_EDUCACIONPADRE
    ['no info', 'Ninguno', 'No sabe', 'No Aplica', 'Primaria incompleta', 'Primaria completa',
     'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
     'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa', 'Postgrado']  # FAMI_EDUCACIONMADRE
]



In [ ]:
codificador = OrdinalEncoder(categories=categories)
train[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE']] = codificador.fit_transform(
    train[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
           'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE',
           'FAMI_EDUCACIONMADRE']]
       )

In [ ]:
train.shape

(296786, 32)

Así queda entonces el Dataset con la codificación de las columnas ordinales.

In [ ]:
train.head()

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,...,"(ingenieria de sistemas,)","(ingenieria electronica,)","(ingenieria industrial,)","(ingenieria mecanica,)","(licenciatura en pedagogia infantil,)","(medicina,)","(negocios internacionales,)","(otros,)","(psicologia,)","(trabajo social,)"
0,0,550236,20183,BOLIVAR,2.0,2.0,4.0,Si,9.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,98545,20203,ANTIOQUIA,5.0,4.0,3.0,Si,7.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,499179,20212,BOGOTÁ,4.0,1.0,4.0,Si,6.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,782980,20195,SUCRE,4.0,4.0,2.0,No,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,785185,20212,ATLANTICO,5.0,3.0,3.0,Si,7.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train['ESTU_PAGOMATRICULAPROPIO'] = train['ESTU_PAGOMATRICULAPROPIO'].fillna('no info')
train['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
No,163869
Si,130110
no info,2807


In [ ]:
print(f"Cantidad nulos tiene internet = {train['ESTU_PAGOMATRICULAPROPIO'].isna().sum()}")
train['FAMI_TIENEINTERNET'].value_counts()

Cantidad nulos tiene internet = 0


,count
FAMI_TIENEINTERNET,
Si,254075
No,31172


## Conversión One-Hot de las columnas FAMI_TIENEINTERNET y ESTU_PAGOMATRICULAPROPIO

In [ ]:
one_hot_encoded = pd.get_dummies(train['ESTU_PAGOMATRICULAPROPIO'], prefix = 'ESTU_PAGOMATRICULAPROPIO')
one_hot_encoded = one_hot_encoded.astype(int)
one_hot_encoded

,ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si,ESTU_PAGOMATRICULAPROPIO_no info
0,0,1,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
296781,0,1,0
296782,1,0,0
296783,1,0,0
296784,1,0,0


In [ ]:
train = pd.concat([train, one_hot_encoded], axis = 1)
train = train.drop('ESTU_PAGOMATRICULAPROPIO', axis = 1)

In [ ]:
train['ESTU_PAGOMATRICULAPROPIO_No'].head()

,ESTU_PAGOMATRICULAPROPIO_No
0,0
1,1
2,1
3,1
4,1


In [ ]:
print(train.columns)

Index([                            'Unnamed: 0',
                                           'ID',
                                      'PERIODO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                           'FAMI_TIENEINTERNET',
                          'FAMI_EDUCACIONPADRE',
                          'FAMI_EDUCACIONMADRE',
                ('administracion de empresas',),
       ('administracion en salud ocupacional',),
                 ('administracion financiera',),
                              ('arquitectura',),
                       ('comunicacion social',),
                        ('contaduria publica',),
                                   ('derecho',),
                                  ('economia',),
                                ('enfermeria',),
                      ('ingenieria ambiental',),
                    

In [ ]:
one_hot_encoded = pd.get_dummies(train['FAMI_TIENEINTERNET'], prefix = 'FAMI_TIENEINTERNET')
one_hot_encoded = one_hot_encoded.astype(int)
one_hot_encoded

,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si
0,0,1
1,0,1
2,0,1
3,1,0
4,0,1
...,...,...
296781,0,1
296782,0,1
296783,0,1
296784,0,1


In [ ]:
train = pd.concat([train, one_hot_encoded], axis = 1)
train = train.drop('FAMI_TIENEINTERNET', axis = 1)

In [ ]:
train['FAMI_TIENEINTERNET_Si'].head()

,FAMI_TIENEINTERNET_Si
0,1
1,1
2,1
3,0
4,1


### Procesamiento de la columna Departamento

Separamos los departamentos con más y menos incidencia de la pobreza monetaria según los datos DANE. Esto porque consideramos que es un factor importante a la hora de la calidad educativa en todas las regiones de Colombia.

In [ ]:
richest_deps = ['BOGOTÁ', 'RISARALDA', 'SANTANDER', 'ANTIOQUIA', 'CALDAS', 'CUNDINAMARCA', 'META', 'VALLE', 'QUINDIO', 'ATLANTICO', 'BOYACA']

In [ ]:
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].apply(lambda x: 1 if x in richest_deps else 0)

In [ ]:
train['ESTU_PRGM_DEPARTAMENTO'].values

array([0, 1, 1, ..., 1, 1, 1])

In [ ]:
train.columns

Index([                            'Unnamed: 0',
                                           'ID',
                                      'PERIODO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                          'FAMI_EDUCACIONPADRE',
                          'FAMI_EDUCACIONMADRE',
                ('administracion de empresas',),
       ('administracion en salud ocupacional',),
                 ('administracion financiera',),
                              ('arquitectura',),
                       ('comunicacion social',),
                        ('contaduria publica',),
                                   ('derecho',),
                                  ('economia',),
                                ('enfermeria',),
                      ('ingenieria ambiental',),
                          ('ingenieria civil',),
                    

In [ ]:
train = train.drop('ESTU_HORASSEMANATRABAJA', axis = 1)

In [ ]:
train = train.drop('Unnamed: 0', axis = 1)

In [ ]:
train = train.drop('PERIODO', axis = 1)

In [ ]:
train.to_csv('test3.csv', index = False)

In [ ]:
train.shape

(296786, 32)

In [ ]:
!head test3.csv

ID,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,"('administracion de empresas',)","('administracion en salud ocupacional',)","('administracion financiera',)","('arquitectura',)","('comunicacion social',)","('contaduria publica',)","('derecho',)","('economia',)","('enfermeria',)","('ingenieria ambiental',)","('ingenieria civil',)","('ingenieria de sistemas',)","('ingenieria electronica',)","('ingenieria industrial',)","('ingenieria mecanica',)","('licenciatura en pedagogia infantil',)","('medicina',)","('negocios internacionales',)","('otros',)","('psicologia',)","('trabajo social',)",ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si,ESTU_PAGOMATRICULAPROPIO_no info,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si
550236,0,2.0,4.0,9.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1,0,0,1
98545,1,5.0,3.0,7.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0